In [ ]:
#| default_exp squeeze

# Squeeze your notebooks 

> How it works 

A first trial... 

In [ ]:
from nbsqueeze import find_pngs, make_img_dir, export_pngs, replace_strings 

In [ ]:
cell_pngs = find_pngs('example_notebook.ipynb')
img_dir = make_img_dir('example_notebook.ipynb')
url_list = export_pngs(cell_pngs, img_dir) 
replace_strings('example_notebook.ipynb', url_list, overwrite=False)


Writing to: example_notebook.ipynb_squeezed.ipynb...
127.0 Kb -> 2.0 Kb


In [ ]:
#|export 

import nbformat 
import re 
import hashlib 
import ipynb_path
import base64 
import os 

In [ ]:
#|export 

def find_pngs(notebook_file): 
    '''Find cells with base64 image/png strings in `notebook_file`.
    
    Returns: [[cell_idx, md5, base64_string], ...]'''
    
    nb = nbformat.read(notebook_file, as_version=nbformat.NO_CONVERT) 
    cells = nb['cells']
    
    cell_pngs = []

    for i, cell in enumerate(cells):   
        
        # 
        if cell['cell_type'] == 'code':  

            # extract base64 png strings 
            # not sure if multiple png strings can exist in outputs... 
            
            code_cell_outputs = cell['outputs'] 
            
            png_strings = [] 
             
            for output in code_cell_outputs: 
                # simple inline image 
                try: 
 
                    base64_string = output['data']['image/png']
                    png_strings.append(base64_string) 
            
                # interactive matplotlib figure
                except: 
                    try: 
                        html_string = output['data']['text/html']
                        m = re.match('.*base64,\s*([^"]+)', html_string)
                        base64_string = m.groups()[0]
                        png_strings.append(base64_string) 
                        
                    except: 
                        pass
                
            if len(png_strings) > 0:
                if len(png_strings) > 1: 
                    print(f'Found multiple png strings in output of cell {i}.')
                    print(' Extracting only first png string!')
                png_string = png_strings[0] 
                
                md5 = hashlib.md5(png_string.encode()).hexdigest()
                md5 = md5[0:10]
                cell_pngs.append([i, md5, png_string]) 
                
                                            
    return cell_pngs 


def make_img_dir(notebook_file): 
    '''Create image subdirectory in current directory for `notebook_file`. '''
    
    img_dir = re.sub('\.ipynb$', '', notebook_file) + '_images' 
    os.makedirs(img_dir, exist_ok=True)
                          
    img_dir = os.path.basename(img_dir)
    
    return img_dir 
   

def export_pngs(cell_pngs, img_dir):
    '''Save base64 png strings from `cell_pngs` as png images in `img_dir`.'''
        
    url_list = []                      

    for idx, md5, base64_string in cell_pngs: 
        
        decoded = base64.b64decode((base64_string))
        png_filepath = os.path.join(img_dir, f'{md5}.png')
                          
        url = [idx, f'./{img_dir}/{md5}.png']
        url_list.append(url)
        
        with open(png_filepath, 'wb') as fh: 
            
            fh.write(decoded)
            fh.close()
                          
    return url_list
                          

def replace_strings(notebook_file, url_list, overwrite=True): 
    ''''''

    nb = nbformat.read(notebook_file, as_version=nbformat.NO_CONVERT) 

    # replace pngs strings in specific cell outputs with links 
    for i, url in url_list: 
        nb['cells'][i].outputs = [nbformat.from_dict({'output_type': 'execute_result', 
                                   'data': {'text/html': ['..', f'<img src=\"{url}\">']}, 
                                   'execution_count': None, 'metadata': {}})]
        
        
    old_size = os.path.getsize(notebook_file) 
    
    if not overwrite: 
        notebook_file = f'{notebook_file}_squeezed.ipynb'
        
    print(f'Writing to: {notebook_file}...')
    
    with open(notebook_file, 'w') as fh: 
        nbformat.write(nb, fh)
    
    new_size = os.path.getsize(notebook_file) 
    
    print(f'{old_size//1e3} Kb -> {new_size//1e3} Kb')
   